# Tratamento dos Dados

In [4]:
print('teste')
variavel = 'oi'
print(variavel)

teste
oi


In [6]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [7]:
# Carregar os dados
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

# Identificar colunas a serem removidas # Remover colunas inúteis
columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df = df.drop(columns=columns_to_drop)
df = df.dropna()

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
colunas_cat = ['Pclass','Sex','Embarked']
for coluna in colunas_cat:
    df[coluna] = df[coluna].astype('O')

colunas_binarias = []
for coluna in df.columns:
    if df[coluna].dtype == 'O':
        categorias = df[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_binarias.append(coluna)

3 niveis: Pclass => [3 1 2]
2 niveis: Sex => ['male' 'female']
3 niveis: Embarked => ['S' 'C' 'Q']


In [4]:
x = df.drop('Survived', axis=1)
y = df['Survived']

x_inteiro = x
y_inteiro = y

#colunas_binarias.remove('Survived')

In [5]:
# DUMMYRISAÇÃO
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_binarias), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

########################################################################################
# PADRONIZAÇÃO DOS DADOS
from sklearn.preprocessing import MinMaxScaler

normalizacao = MinMaxScaler()
#x = normalizacao.fit_transform(x)

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 
x.head()

,onehotencoder__Pclass_1,onehotencoder__Pclass_2,onehotencoder__Pclass_3,onehotencoder__Sex_male,onehotencoder__Embarked_C,onehotencoder__Embarked_Q,onehotencoder__Embarked_S,remainder__Age,remainder__SibSp,remainder__Parch,remainder__Fare
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,22.0,1.0,0.0,7.2500
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,38.0,1.0,0.0,71.2833
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,26.0,0.0,0.0,7.9250
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0,1.0,0.0,53.1000
4,0.0,0.0,1.0,1.0,0.0,0.0,1.0,35.0,0.0,0.0,8.0500


In [6]:
# SEPARANDO DADOS PARA TREINO E TESTE
from sklearn.model_selection import train_test_split
x, x_teste, y, y_teste = train_test_split(x_inteiro, y_inteiro, 
                                          #stratify = y, #para manter a proporção da Var Dep nos splits
                                          random_state = 5, #raiz da aleatoridade
                                          test_size = 0.25) #porcentagem que ira ser separado para testes

x.shape, x_teste.shape, y.shape, y_teste.shape

((534, 7), (178, 7), (534,), (178,))

# FUNÇÕES

In [7]:
# CODIGOS PARA VALIDAÇÃO
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay

def avaliar_modelo(y_verdadeiro, y_predito, conjunto_nome):
    """
    Função para avaliar e imprimir o relatório de classificação e a matriz de confusão.
    """
    #if conjunto_nome == "Treino":
     #   pass
    #else:
    print('*' * 70)
    print("Relatório de Classificação para o Conjunto de", conjunto_nome,":\n")
    print(classification_report(y_verdadeiro, y_predito))
    
    print('*' * 55)
    
    print("Matriz de Confusão para o Conjunto de", conjunto_nome,":\n")
    print(confusion_matrix(y_verdadeiro, y_predito))
    print('*' * 70)

    display(RocCurveDisplay.from_predictions(y_verdadeiro, y_predito, name = conjunto_nome))

In [8]:
def intervalo_conf(resultados):
    media = resultados.mean()
    desvio_padrao = resultados.std()
    ic_0 = round(media - 2*desvio_padrao, 2)
    ic_1 = round(min(media + 2*desvio_padrao, 1), 2)
    ic = '[' + str(ic_0) + ' - ' + str(ic_1) + ']'
    return ic

def histograma(dados):
    dados = dados
    plt.figure(figsize=(15,5))
    plt.hist(dados, edgecolor='black', density=True) #bins=q_bins, 

    plt.title('hist', fontsize=15)
    plt.grid(True, color='gray')

    # Adicionar linhas verticais para média e mediana
    plt.axvline(x = dados.mean(), color='red', linestyle='--', label='Média')
    plt.axvline(x = dados.median(), color='blue', linestyle='--', label='Mediana')

    # Adicionar legenda personalizada
    texto_count = 'Count = ' + str(round(len(dados), 0))
    texto_media = 'Média = '+ str(round(dados.mean(), 2))
    texto_dp = 'DP = '+ str(round(dados.std(), 2))
    texto_min = 'Min = '+ str(round(dados.min(), 2))
    texto_Q1 = 'Q1 = ' + str(round(dados.quantile(0.25), 2))
    texto_mediana = 'Q2 = '+ str(round(dados.median(), 2))
    texto_Q3 = 'Q3 = ' + str(round(dados.quantile(0.75), 2))
    texto_max = 'Max = '+ str(round(dados.max(), 2))
    ic = 'IC ' + intervalo_conf(dados)
    texto_legenda = '\n'.join([texto_count, 
                               texto_min,
                               texto_media, texto_dp, 
                               texto_Q1, texto_mediana, texto_Q3,
                               texto_max, 
                               ic])

    plt.text(0.99, 0.96, texto_legenda, ha='right', va='top', transform=plt.gca().transAxes,
             bbox=dict(facecolor='black', edgecolor='gray', boxstyle='round'),
             fontsize=12)

    plt.show()

# Modelos de Machine Learning

## LazyClassifier

LazyPedict é uma biblioteca que ajuda a semi-automatizar os modelos de Machine Learning.

Essa biblioteca constrói muitos modelos básicos sem muito código e ajuda a entender quais modelos funcionam melhor sem qualquer ajuste de parâmetro.

Após a obtenção dos resultados de acurácia e avaliação das métricas podemos escolher os melhores modelos e aplicar o ajuste de hiperparâmetros a eles.

Então, no caso dos dados deste projeto vou usar o LazyClassifier para tentar resolver o problema de classificação:

In [16]:
from lazypredict.Supervised import LazyClassifier, LazyRegressor

np.random.seed(73246) #para garantir a reprodutibilidade

#x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=0)

# fit all models
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(x, x_teste, y, y_teste) 
'''"clf" está retornando dois valores, Modelo e Previsão, enquanto modelo significa 
todos os modelos e com algumas métricas e previsão significa todo o valor previsto que é ŷ'''
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 13.61it/s]

[LightGBM] [Info] Number of positive: 224, number of negative: 310
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 184
[LightGBM] [Info] Number of data points in the train set: 534, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.419476 -> initscore=-0.324926
[LightGBM] [Info] Start training from score -0.324926
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.83,0.83,0.83,0.83,0.27
XGBClassifier,0.82,0.82,0.82,0.82,0.14
SVC,0.84,0.82,0.82,0.84,0.04
NuSVC,0.84,0.82,0.82,0.84,0.04
LabelPropagation,0.82,0.80,0.80,0.82,0.05
LabelSpreading,0.82,0.80,0.80,0.82,0.03
KNeighborsClassifier,0.83,0.80,0.80,0.83,0.04
LogisticRegression,0.82,0.80,0.80,0.82,0.03
RandomForestClassifier,0.81,0.80,0.80,0.81,0.24
